In [ ]:
import numpy as np
from tqdm import trange
import os
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch import nn
from tqdm import tqdm

import mowl
mowl.init_jvm('10g')
from mowl.datasets import PathDataset, Dataset
from mowl.base_models import EmbeddingELModel
from mowl.nn import ELEmModule, ELBoxModule, BoxSquaredELModule
from mowl.utils.data import FastTensorDataLoader

from src.utils import *
from src.elmodule import *

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

In [ ]:
#dataset_name = 'family'
#dataset_name = 'pizza'
#dataset_name = 'OWL2DL-1'
dataset_name = 'lubm'

In [ ]:
experiments = get_experimets(dataset_name)

**Models**

**Eval**

**Experiments: Box2EL**

In [ ]:
for experiment in experiments: 
    dataset_name = experiment['dataset_name']
    file_name = experiment['file_name']
    format_ = experiment['format_']
    add_noise = experiment['add_noise']                                                                                                                   
   
    dataset = PathDataset(ontology_path=f'datasets/bin/{file_name}_train.owl',
                          testing_path=f'datasets/bin/{dataset_name}_test.owl',
                          validation_path=f'datasets/bin/{dataset_name}_val.owl')
    
    train_ont = preprocess_ontology_el(dataset.ontology)
    test_ont = preprocess_ontology_el(dataset.testing)
    valid_ont = preprocess_ontology_el(dataset.validation)

    dataset = Dataset(train_ont, testing=test_ont, validation=valid_ont)
    
    box2el = ElModel(dataset, 
                     module_name='box2el', 
                     dim=200, 
                     margin=0.1, 
                     batch_size=4096*8, 
                     test_batch_size=32, 
                     epochs=500, 
                     learning_rate=0.001,
                     device=device)
    
    box2el._train()
    
    print(f'{file_name}:')
    print('Membership:')
    box2el._eval('membership')
    print('Subsumption:')
    box2el._eval('subsumption')
    print()